# 股票价格预测实验笔记本

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from src.data_loader import load_stock_data
from src.feature_engineer import create_sequences
from models.model import StockPriceLSTM

# GPU配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

In [ ]:
# 数据加载与预处理
data_path = 'data/raw/stock_prices.csv'
df = load_stock_data(data_path)

# 特征工程
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close']])

# 时序数据准备
sequence_length = 60
X, y = create_sequences(scaled_data, sequence_length)

# 划分训练集和测试集
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 转换为Tensor
X_train = torch.FloatTensor(X_train).to(device)
y_train = torch.FloatTensor(y_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_test = torch.FloatTensor(y_test).to(device)

In [ ]:
# 模型初始化
input_dim = 1
hidden_dim = 50
num_layers = 2
output_dim = 1

model = StockPriceLSTM(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    output_dim=output_dim
).to(device)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 模型训练
epochs = 100
batch_size = 32

for epoch in range(epochs):
    model.train()
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [ ]:
# 模型评估
model.eval()
with torch.no_grad():
    test_predictions = model(X_test)
    test_loss = criterion(test_predictions, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# 反归一化
test_predictions = scaler.inverse_transform(test_predictions.cpu())
y_test_original = scaler.inverse_transform(y_test.cpu())

# 可视化结果
plt.figure(figsize=(12,6))
plt.plot(y_test_original, label='实际价格')
plt.plot(test_predictions, label='预测价格')
plt.title('股票价格预测')
plt.legend()
plt.show()